# Bank Marketing Analysis - Funktionale Implementierung

**Autoren:** Peter Ngo, Alex Uscata  
**Klasse:** INA 23A  
**Modul:** M323 - Funktionale Programmierung  
**Datum:** 2026-01-06  
**Version:** 2.1.0

Dieses Notebook demonstriert die funktionale Analyse von Bank-Marketing-Daten mit Schwerpunkt auf:
- **Funktionskomposition** (`compose()`, `pipe()`)
- **Higher-Order Functions** (`map()`, `filter()`, `reduce()`)
- **Immutability** und Pure Functions
- **Deklarative Datentransformationen**
- **Prädikat-Komposition** für flexible Filterung

## Setup & Daten laden

In [ ]:
import sys
import os
import math

# Pfad zum src-Ordner hinzufügen
sys.path.insert(0, os.path.abspath('../src'))

# Imports der funktionalen Version
from functional_version import (
    # Utility functions
    _header, _table, _fmt_pct, _fmt_num, _fmt_int,
    
    # Core analysis functions
    _apply_filters, _success_overall, _duration_stats, 
    _duration_buckets, _group_metrics, _marital_compare,
    _compare_two_groups, _anova_f_balance, 
    
    # Statistical functions
    _mean, _variance_population,
    
    # Higher-order functions
    compose, pipe, create_balance_filter, combine_predicates,
    
    # Grouping functions
    _group_by_key
)
from common_io import load_bank_data

# Daten laden
data = load_bank_data()
current = list(data)

print(f"✓ Datensätze geladen: {len(data)}")
print(f"✓ Funktionale Version V2.0 geladen")
print(f"✓ Neue Features: compose(), pipe(), Prädikat-Komposition")

---
## Demo: Funktionskomposition mit `compose()` und `pipe()`

Bevor wir mit der Datenanalyse beginnen, demonstrieren wir die neuen Higher-Order Functions für Funktionskomposition.

### `compose()` - Rechts-nach-Links (Mathematische Notation)
Mathematische Notation: `(f ∘ g)(x) = f(g(x))`

### `pipe()` - Links-nach-Rechts (Unix-Pipe-Style)
Unix-Style: `data | fn1 | fn2 | fn3`

In [ ]:
print(_header("FUNKTIONSKOMPOSITION DEMO"))

# Einfache Transformationen definieren
add_10 = lambda x: x + 10
multiply_2 = lambda x: x * 2
square = lambda x: x ** 2

# compose: rechts-nach-links (mathematisch)
# Liest sich: square(multiply_2(add_10(5)))
transform_compose = compose(square, multiply_2, add_10)
result1 = transform_compose(5)
print(f"compose(square, multiply_2, add_10)(5)")
print(f"  = square(multiply_2(add_10(5)))")
print(f"  = square(multiply_2(15))")
print(f"  = square(30)")
print(f"  = {result1}")

print()

# pipe: links-nach-rechts (intuitiv)
# Liest sich: 5 |> add_10 |> multiply_2 |> square
transform_pipe = pipe(add_10, multiply_2, square)
result2 = transform_pipe(5)
print(f"pipe(add_10, multiply_2, square)(5)")
print(f"  = 5 |> add_10 |> multiply_2 |> square")
print(f"  = 15 |> multiply_2 |> square")
print(f"  = 30 |> square")
print(f"  = {result2}")

print()
print("✓ Beide ergeben das gleiche Resultat (900)")
print("✓ compose: Mathematische Notation, rechts-nach-links")
print("✓ pipe: Unix-Style, links-nach-rechts (intuitiver)")

---
## Demo: Prädikat-Komposition

Demonstration von `combine_predicates()` für deklarative Filterbedingungen.

In [ ]:
print(_header("PRÄDIKAT-KOMPOSITION"))

# Einzelne Prädikate definieren
has_housing = lambda r: r.get('housing') is True
has_loan = lambda r: r.get('loan') is True  
high_balance = lambda r: r.get('balance', 0) > 1000

# Prädikate kombinieren (AND-Logik)
combined = combine_predicates(has_housing, has_loan, high_balance)

# Test-Datensätze
test_records = [
    {'housing': True, 'loan': True, 'balance': 1500, 'name': 'Record 1'},
    {'housing': True, 'loan': False, 'balance': 2000, 'name': 'Record 2'},
    {'housing': False, 'loan': True, 'balance': 500, 'name': 'Record 3'},
    {'housing': True, 'loan': True, 'balance': 800, 'name': 'Record 4'},
]

# Filter anwenden
filtered = list(filter(combined, test_records))

print("Test-Datensätze:")
for r in test_records:
    passed = "✓" if combined(r) else "✗"
    print(f"  {passed} {r['name']}: housing={r['housing']}, loan={r['loan']}, balance={r['balance']}")

print(f"\nGefiltert: {len(filtered)} von {len(test_records)} Datensätzen")
print(f"✓ Prädikate sind composable und wiederverwendbar")
print(f"✓ Deklarativ: Definiere WAS, nicht WIE")

---
## Demo: Pipeline-Komposition mit echten Bank-Daten

Zeigt wie `pipe()` für komplexe Datenverarbeitungs-Pipelines genutzt werden kann.

In [ ]:
print(_header("PIPELINE-KOMPOSITION MIT BANK-DATEN"))

# Beispiel: Durchschnittsalter von Kunden mit Housing Loan und Balance > 1000

# Pipeline-Schritte definieren
filter_housing = lambda data: filter(lambda r: r.get('housing') is True, data)
filter_high_balance = lambda data: filter(lambda r: r.get('balance', 0) > 1000, data)
extract_age = lambda data: map(lambda r: r.get('age', 0), data)
to_list_fn = lambda it: list(it)
calculate_average = lambda ages: sum(ages) / len(ages) if ages else 0

# Pipeline zusammenbauen (liest sich wie natürliche Sprache)
analyze_customers = pipe(
    filter_housing,
    filter_high_balance,
    extract_age,
    to_list_fn,
    calculate_average
)

# Pipeline auf echte Daten anwenden (erste 1000 Datensätze für Demo)
sample_data = list(data)[:1000]
avg_age = analyze_customers(sample_data)

print(f"Durchschnittsalter von Kunden mit:")
print(f"  - Housing Loan = True")
print(f"  - Balance > 1000")
print(f"\nResultat: {avg_age:.1f} Jahre")

print(f"\n✓ Pipeline ist lesbar wie eine Spezifikation")
print(f"✓ Jeder Schritt ist einzeln testbar")
print(f"✓ Keine Mutation, keine Schleifen - rein deklarativ")

---
---
# DATENANALYSE

Die folgenden Optionen führen verschiedene Analysen auf dem Bank-Marketing-Datensatz durch.
Alle Funktionen verwenden funktionale Programmierkonzepte.

---
## Option 1: Erfolgsquote gesamt

Zeigt die Gesamterfolgsquote der Marketingkampagne.

In [ ]:
print(_header("ERFOLGSQUOTE"))
total, yes_count, quote = _success_overall(current)
rows = [
    ["Total", str(total)],
    ["Yes", str(yes_count)],
    ["Quote", _fmt_pct(quote)]
]
print(_table(["Metric", "Value"], rows, aligns=["<", ">"]))

---
## Option 2: Filter setzen

Filtert die Daten nach housing, loan und balance-Kriterien.

**Anleitung:** Passe die Werte unten an:
- `housing_filter`: `True` für yes, `False` für no, `None` für keinen Filter
- `loan_filter`: `True` für yes, `False` für no, `None` für keinen Filter
- `balance_gt_filter`: Mindestwert für balance oder `None`

In [ ]:
# === PARAMETER ANPASSEN ===
housing_filter = True      # None, True oder False
loan_filter = False         # None, True oder False
balance_gt_filter = None   # None oder z.B. 1000

# Neue Methode: Prädikat-Komposition (funktionaler Ansatz)
predicates = []

if housing_filter is not None:
    predicates.append(lambda r, h=housing_filter: r.get("housing") is h)

if loan_filter is not None:
    predicates.append(lambda r, l=loan_filter: r.get("loan") is l)

if balance_gt_filter is not None:
    predicates.append(create_balance_filter(balance_gt_filter))

# Prädikate mit AND-Logik kombinieren
if predicates:
    combined_filter = combine_predicates(*predicates)
    current = list(filter(combined_filter, data))
else:
    current = list(data)

print(_header("FILTER RESULT (Prädikat-Komposition)"))
print(f"Aktueller Datenbestand: {len(current)} / {len(data)}")
print(f"\nAngewendete Filter:")
print(f"  housing = {housing_filter}")
print(f"  loan = {loan_filter}")
print(f"  balance > {balance_gt_filter}")
print(f"\n✓ Funktional: combine_predicates() verwendet")
print(f"✓ Deklarativ: Prädikate composable und wiederverwendbar")

---
## Option 3: Transformationen

Berechnet Statistiken für transformierte Balance-Werte:
- `log(balance)` - Logarithmische Transformation
- `balance^2+1` - Quadratische Transformation

In [ ]:
print(_header("TRANSFORMATIONEN (mit pipe)"))

# Definiere wiederverwendbare Pipeline-Komponenten
filter_valid_balance = lambda records: filter(
    lambda r: isinstance(r.get("balance"), (int, float)), 
    records
)
extract_balances = lambda records: map(
    lambda r: float(r["balance"]), 
    records
)
to_list = lambda it: list(it)

# Pipeline: filter -> extract -> collect
balance_pipeline = pipe(
    filter_valid_balance,
    extract_balances,
    to_list
)

balances = balance_pipeline(current)

# Log-Transformations-Pipeline
log_transform_pipeline = pipe(
    lambda vals: map(lambda b: math.log(b) if b > 0.0 else None, vals),
    lambda vals: filter(lambda x: x is not None, vals),
    to_list
)

# Quadrat + 1 Transformations-Pipeline
square_plus_one_pipeline = pipe(
    lambda vals: map(lambda b: b * b + 1.0, vals),
    to_list
)

logs = log_transform_pipeline(balances)
sq1 = square_plus_one_pipeline(balances)

def stats_line(name, values):
    mu = _mean(values)
    var = _variance_population(values)
    mn = min(values) if values else None
    mx = max(values) if values else None
    return [name, str(len(values)), _fmt_num(mn), _fmt_num(mx), _fmt_num(mu), _fmt_num(var)]

rows = [
    stats_line("log(balance)", list(map(float, logs))),
    stats_line("balance^2+1", sq1)
]
print(_table(["Transform", "n", "min", "max", "mean", "var"], rows, aligns=["<", ">", ">", ">", ">", ">"]))

print(f"\n✓ Funktional: pipe() für deklarative Pipelines")
print(f"✓ Liest sich wie Unix-Pipes: data | filter | map | collect")
print(f"✓ Komponenten sind wiederverwendbar und testbar")

---
## Option 4a: Duration Analyse

Statistische Auswertung der Gesprächsdauer (duration).

In [ ]:
print(_header("DURATION ANALYSE"))
mn, mx, mu, var = _duration_stats(current)
rows = [[_fmt_int(mn), _fmt_int(mx), _fmt_num(mu), _fmt_num(var)]]
print(_table(["min", "max", "mean", "var"], rows, aligns=[">", ">", ">", ">"]))

## Option 4b: Duration Buckets

Gruppierung der Duration in Intervalle mit Erfolgsquoten.

**Anleitung:** Passe `bucket_size` an (z.B. 60 für 60 Sekunden-Intervalle).

In [ ]:
# === PARAMETER ANPASSEN ===
bucket_size = 60  # Intervallgröße in Sekunden

buckets = _duration_buckets(current, bucket_size)
b_rows = list(map(lambda t: [t[0], str(t[1]), _fmt_pct(t[2])], buckets))

print(_header("DURATION BUCKETS"))
print(_table(["bucket", "count", "success"], b_rows, aligns=["<", ">", ">"]))

---
## Option 5: Group by Education

Gruppiert die Daten nach Bildungsniveau und zeigt Durchschnittswerte.

In [ ]:
print(_header("GROUP BY EDUCATION"))
metrics = _group_metrics(current, "education")
rows = list(
    map(
        lambda t: [t[0] or "(blank)", str(t[1]), _fmt_num(t[2], 1), _fmt_num(t[3], 2), _fmt_pct(t[4])],
        metrics,
    )
)
print(_table(["education", "count", "avg(age)", "avg(balance)", "success"], rows, aligns=["<", ">", ">", ">", ">"]))

anova = _anova_f_balance(current, "education")
print()
if anova is None:
    print("ANOVA F-Wert: n/a")
else:
    f_value, df_b, df_w = anova
    print(f"ANOVA F-Wert: {_fmt_num(f_value, 2)} (df={df_b},{df_w})")
    

---
## Option 6: Group by Marital

Gruppiert die Daten nach Familienstand (single, married, divorced).

In [ ]:
print(_header("GROUP BY MARITAL"))
metrics = _marital_compare(current)
rows = list(map(lambda t: [t[0], str(t[1]), _fmt_num(t[2], 2), _fmt_num(t[3], 1), _fmt_pct(t[4])], metrics))
print(_table(["marital", "count", "avg(balance)", "avg(duration)", "success"], rows, aligns=["<", ">", ">", ">", ">"]))

---
## Option 7: Vergleich zweier Gruppen

Vergleicht zwei spezifische Gruppen innerhalb eines Feldes.

**Anleitung:**
1. Wähle das Feld: `education`, `marital` oder `job`
2. Wähle zwei Gruppen zum Vergleichen

In [ ]:
# === PARAMETER ANPASSEN ===
field = "education"  # "education", "marital" oder "job"
group_a = "tertiary"  # Erste Gruppe
group_b = "secondary" # Zweite Gruppe

# Verfügbare Gruppen anzeigen
available = sorted({(r.get(field) or "") for r in current})
print(f"Verfügbare Gruppen in '{field}': {', '.join(available)}")
print()

# Vergleich durchführen
print(_header("VERGLEICH ZWEIER GRUPPEN"))
m1, m2 = _compare_two_groups(current, field, group_a, group_b)

def row(m):
    name, cnt, avg_age, avg_bal, avg_dur, rate = m
    return [name or "(blank)", str(cnt), _fmt_num(avg_age, 1), _fmt_num(avg_bal, 2), _fmt_num(avg_dur, 1), _fmt_pct(rate)]

rows = [row(m1), row(m2)]
print(_table([field, "count", "avg(age)", "avg(balance)", "avg(duration)", "success"], rows, aligns=["<", ">", ">", ">", ">", ">"]))

delta = m1[-1] - m2[-1]
sign = "+" if delta >= 0 else ""
print(f"\nΔ Erfolgsquote (A-B): {sign}{delta * 100:0.1f}%")

---
## Option 8: ANOVA-ähnlicher F-Wert

Berechnet einen F-Wert für die Balance-Unterschiede zwischen Gruppen.

**Anleitung:** Wähle das Gruppierungsfeld (`education`, `marital` oder `job`).

In [ ]:
# === PARAMETER ANPASSEN ===
field = "education"  # "education", "marital" oder "job"

print(_header("ANOVA-ÄHNLICHER F-WERT (BALANCE)"))
result = _anova_f_balance(current, field)

if result is None:
    print("Nicht genug Daten für F-Berechnung (mind. 2 Gruppen, ausreichend Beobachtungen).")
else:
    f_value, dfb, dfw = result
    f_text = "inf" if math.isinf(f_value) else f"{f_value:0.3f}"
    print(f"F({dfb}, {dfw}) = {f_text}")
    print()
    if math.isinf(f_value):
        print("Interpretation: Innerhalb-Varianz ist 0; Gruppenmittelwerte unterscheiden sich stark")
        print("oder Werte sind konstant pro Gruppe.")
    elif f_value < 1.5:
        print("Interpretation: Eher geringe Unterschiede der Mittelwerte zwischen Gruppen")
        print("(relativ zur Streuung).")
    elif f_value < 5.0:
        print("Interpretation: Moderate Unterschiede der Mittelwerte zwischen Gruppen.")
    else:
        print("Interpretation: Deutliche Unterschiede der Mittelwerte zwischen Gruppen möglich")
        print("(hoher F-Wert).")

---
## Filter zurücksetzen

Setzt den Datensatz auf die ursprünglichen Daten zurück.

In [ ]:
current = list(data)
print(f"✓ Filter zurückgesetzt: {len(current)} Datensätze aktiv")